In [ ]:
# 구글 드라이브 접근 코드
from google.colab import drive
drive.mount('/content/drive')
My_path = '/content/drive/MyDrive/LG_Academy/AI_Code/'

## 라이브러리 import

In [ ]:
import pandas            as pd
import numpy             as np
import seaborn           as sns
import scipy.stats       as sp
import matplotlib.pyplot as plt

.

.

.



## 데이터 불러오기

In [ ]:
FeatureData = pd.read_csv(My_path + 'ProcessedData/FeatureData.csv', sep=',', header=None)
FeatureData.shape

특징데이터 정상/고장 분리

In [ ]:
NoOfData = int(FeatureData.shape[0]/2)
Normal_FeatureData   = FeatureData.iloc[:NoOfData, :]
Abnormal_FeatureData = FeatureData.iloc[NoOfData:, :]

print(Normal_FeatureData.shape)
print(Abnormal_FeatureData.shape)

.

.

.



## P-value 계산

In [ ]:
NoOfFeature = FeatureData.shape[1] # 추출된 Feature 갯수

P_value = np.zeros((NoOfFeature , 2))

# 특징값 각각 T-검정 수행
for i in np.arange(NoOfFeature):

    T_test       = np.array(sp.ttest_ind(Normal_FeatureData.iloc[:, i] , Abnormal_FeatureData.iloc[:, i]))
    P_value[i,0] = int(i)     # Feature Index
    P_value[i,1] = T_test[1]  # P값 (P-value)

P_value      = pd.DataFrame(P_value)
P_value

In [ ]:
P_value_Rank = P_value.sort_values([1],ascending=True)  # P-value 기준 오름차순 정렬

P_value_Rank

In [ ]:
# 특징 이름 추가
Frequency_select = 8

TimeFeatureName = []
FreqFeatureName = []

SensorNames  = ['Acc_x', 'Acc_y']
FeatureNames = ['Max', 'Min', 'Mean', 'RMS', 'Var', 'Skew', 'Kurt', 'Crest', 'Shape', 'Impulse']

for Sensor in SensorNames:
    for Feature in FeatureNames:
        TimeFeatureName.append( '{}_Time_{}'.format(Sensor, Feature) )

for Sensor in SensorNames:
    for i in range(Frequency_select):
        for Feature in FeatureNames:
            FreqFeatureName.append( '{}_D{}_{}'.format(Sensor, i+1, Feature) )

AllFeatureName = TimeFeatureName + FreqFeatureName

In [ ]:
Rank = 10
TopFeatureList = list(P_value_Rank.iloc[:Rank, 0])

for i in range(Rank):
    print(AllFeatureName[int(TopFeatureList[i])])

In [ ]:
# 특징값 저장
path1 = My_path + 'ProcessedData/P_value.csv'        # path = '파일 경로/저장할 파일 이름'
path2 = My_path + 'ProcessedData/P_value_Rank.csv'   # path = '파일 경로/저장할 파일 이름'
P_value.to_csv(path1, sep=',', header=None, index=None)
P_value_Rank.to_csv(path2, sep=',', header=None, index=None)

.

.

.



## PDF(Probabilistic Density Function) 그래프 확인 (정상 vs 고장)

In [ ]:
# P-value 낮은 순위 (구분성 높은 순위)
FeatureRank = 0

plt.figure(figsize = (8,6))
# PDF 그래프
sns.histplot(Normal_FeatureData.iloc[:, int(P_value_Rank.iloc[FeatureRank, 0])],   stat='percent', kde= True, element='step', color = 'b', label = 'Normal')
sns.histplot(Abnormal_FeatureData.iloc[:, int(P_value_Rank.iloc[FeatureRank, 0])], stat='percent', kde= True, element='step', color = 'r', label = 'Abnormal')
plt.legend(loc='upper right')
plt.grid(alpha = 0.4)
plt.show()

## P-value 기반 구분성 상위 특징값 선택

In [ ]:
Rank = 10

Normal   = np.zeros((NoOfData, Rank))
Abnormal = np.zeros((NoOfData, Rank))

for i in range(Rank):

    index         = int(P_value_Rank.iloc[i,0])
    Normal[:, i]   = Normal_FeatureData.iloc[:, index].values
    Abnormal[:, i] = Abnormal_FeatureData.iloc[:, index].values

# 정상, 고장 특징값 합치기
FeatureSelected = pd.DataFrame(np.concatenate([Normal, Abnormal] , axis=0))

print("Selected Feature Data Size :", FeatureSelected.shape)
print("= 데이터 %d개(정상/고장 각 %d개씩)가 각각 %d개의 최종 선택된 특징값으로 구성됨"
      %(FeatureSelected.shape[0], FeatureSelected.shape[0]/2, FeatureSelected.shape[1]))

최종 선택된 특징 데이터 저장 (.csv 파일)

In [ ]:
path = My_path + 'ProcessedData/FeatureSelected.csv'   # path = '파일 경로//저장할 파일 이름'
FeatureSelected.to_csv(path, sep=',', header=None, index=None)